In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/materials-data/user_preferences_dummy_data_with_companies.xlsx
/kaggle/input/materials-data/material_sourcing_dummy_data_with_companies.xlsx


In [2]:
import numpy as np
import pandas as pd

In [3]:
materials_data=pd.read_excel("/kaggle/input/materials-data/material_sourcing_dummy_data_with_companies.xlsx")
user_data=pd.read_excel("/kaggle/input/materials-data/user_preferences_dummy_data_with_companies.xlsx")

In [4]:
materials_data.head(5)

,Location ID,"Geotag (Latitude, Longitude)",Distance to Nearest Supplier (km),Average Transportation Cost ($),Material Type,Current Market Price ($),Historical Price Average ($),Order Lead Time (days),Delivery Schedule Variability (days),Inventory Levels (units),Quality Compliance Rate (%),Defect Rate (%),User Preference Score (1-10),Project Budget ($),Material Requirement (units),Supplier Company
0,Location_1,"(-53.10282682515223, 118.3980117177394)",74.641773,108.882948,Steel,444.147388,548.059625,11,1.350599,859,84.539571,1.208006,2,50172.687699,243,Global Industries
1,Location_2,"(-37.23340855301761, -24.880491481749857)",63.518618,105.851851,Steel,292.592549,461.834910,13,1.263247,59,94.878104,3.406411,9,24561.054317,81,Global Industries
2,Location_3,"(71.34044733380156, -90.46286186845323)",22.862350,103.542252,Pipes,177.112592,471.660031,23,4.565273,511,98.384161,3.579923,2,27300.779113,155,Eco Friendly Materials Co.
3,Location_4,"(-87.65965376806751, 42.17219517745082)",38.761534,104.827054,Steel,241.352683,170.749412,5,2.855257,727,88.475402,3.036622,5,7740.537571,449,Global Industries
4,Location_5,"(-74.60846444619574, 74.4397980787605)",79.460220,101.338140,Pipes,66.906991,336.234711,6,2.280109,507,86.842652,0.442892,8,7025.004346,253,Global Industries


In [5]:
user_data.head(5)

,User ID,Requirements,Expectations,Expected Price ($),Expected Delivery Time (days),Old Product Ratings (1-5),Preferred Location,User Latitude,User Longitude,Material Type Preference,Preferred Supplier Company
0,User_1,High Durability,Immediate Use,352.966497,1,4.385149,Location_388,-52.116885,144.764006,Cement,Global Industries
1,User_2,Low Cost,Immediate Use,701.217507,4,2.425467,Location_84,-32.714100,150.284240,Pipes,Reliable Supplies Ltd.
2,User_3,High Quality,Long-lasting,259.014083,11,4.329995,Location_434,-46.787462,155.038501,Tiles,Quality Materials Inc.
3,User_4,High Quality,Budget-friendly,622.205679,10,4.403114,Location_371,10.246518,-82.639373,Steel,Reliable Supplies Ltd.
4,User_5,High Durability,Long-lasting,913.631488,10,2.695722,Location_48,-70.519204,-35.871986,Steel,Eco Friendly Materials Co.


In [6]:
user_data['User ID'].value_counts().sum()

500

In [7]:
materials_data['Supplier Company'].str.lower().value_counts()

Supplier Company
global industries             117
quality materials inc.         97
reliable supplies ltd.         97
acme corp                      95
eco friendly materials co.     94
Name: count, dtype: int64

In [8]:
user_data['Preferred Supplier Company'].str.lower().value_counts()

Preferred Supplier Company
reliable supplies ltd.        110
global industries             107
eco friendly materials co.    104
acme corp                      99
quality materials inc.         80
Name: count, dtype: int64

In [9]:
materials_data['Supplier Company']=materials_data['Supplier Company'].str.lower()
user_data['Preferred Supplier Company']=user_data['Preferred Supplier Company'].str.lower()

In [10]:
materials_data['Latitude']=materials_data["Geotag (Latitude, Longitude)"].apply(lambda x: eval(x)[0])
materials_data['Longitude']=materials_data["Geotag (Latitude, Longitude)"].apply(lambda x: eval(x)[1])

In [11]:
materials_data.drop(columns=['Geotag (Latitude, Longitude)','Location ID'],inplace=True)
materials_data=materials_data.groupby(['Supplier Company','Material Type']).mean().reset_index()
supplier_loc=materials_data.drop(columns=['Material Type']).groupby(['Supplier Company']).mean().reset_index()[['Supplier Company','Latitude','Longitude']]

In [12]:
for company in supplier_loc['Supplier Company']:
    materials_data.loc[materials_data['Supplier Company']==company,'Latitude']=supplier_loc[supplier_loc['Supplier Company']==company]['Latitude'].iloc[0]
    materials_data.loc[materials_data['Supplier Company']==company,'Longitude']=val=supplier_loc[supplier_loc['Supplier Company']==company]['Longitude'].iloc[0]

In [13]:
final_df=pd.merge(user_data,materials_data,left_on=['Preferred Supplier Company','Material Type Preference'],right_on=['Supplier Company','Material Type'])

In [15]:
final_df.columns

Index(['User ID', 'Requirements', 'Expectations', 'Expected Price ($)',
       'Expected Delivery Time (days)', 'Old Product Ratings (1-5)',
       'Preferred Location', 'User Latitude', 'User Longitude',
       'Material Type Preference', 'Preferred Supplier Company',
       'Supplier Company', 'Material Type',
       'Distance to Nearest Supplier (km)', 'Average Transportation Cost ($)',
       'Current Market Price ($)', 'Historical Price Average ($)',
       'Order Lead Time (days)', 'Delivery Schedule Variability (days)',
       'Inventory Levels (units)', 'Quality Compliance Rate (%)',
       'Defect Rate (%)', 'User Preference Score (1-10)', 'Project Budget ($)',
       'Material Requirement (units)', 'Latitude', 'Longitude'],
      dtype='object')

In [19]:
final_df['Material Type Preference']

0      Cement
1       Pipes
2       Tiles
3       Steel
4       Steel
        ...  
495    Cement
496     Tiles
497     Tiles
498    Cement
499     Tiles
Name: Material Type Preference, Length: 500, dtype: object

In [21]:
final_df.drop(columns=['User ID','Preferred Location','Preferred Supplier Company','Material Type Preference'],inplace=True)

In [22]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 23 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Requirements                          500 non-null    object 
 1   Expectations                          500 non-null    object 
 2   Expected Price ($)                    500 non-null    float64
 3   Expected Delivery Time (days)         500 non-null    int64  
 4   Old Product Ratings (1-5)             500 non-null    float64
 5   User Latitude                         500 non-null    float64
 6   User Longitude                        500 non-null    float64
 7   Supplier Company                      500 non-null    object 
 8   Material Type                         500 non-null    object 
 9   Distance to Nearest Supplier (km)     500 non-null    float64
 10  Average Transportation Cost ($)       500 non-null    float64
 11  Current Market Pric

In [29]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=False)
encoded=enc.fit_transform(final_df[['Requirements','Expectations','Material Type']])

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [32]:
encoded_df = pd.DataFrame(encoded, columns=enc.get_feature_names_out(['Requirements','Expectations','Material Type']))

In [35]:
final_df.drop(columns=['Requirements','Expectations','Material Type'],inplace=True)

In [38]:
final_df=pd.concat([final_df,encoded_df],axis=1)

In [43]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
arr=le.fit_transform(final_df['Supplier Company'])

In [46]:
target=pd.DataFrame(arr, columns=['target'])

In [47]:
final_df.drop(columns=['Supplier Company'],inplace=True)

In [49]:
final_df=pd.concat([final_df,target],axis=1)

In [51]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(final_df.drop(columns='target'), final_df[['target']], test_size=0.33, random_state=42)

In [55]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
model = xgb.XGBClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 1.0


In [76]:
pred=model.predict_proba(X_test.iloc[[2]])

In [91]:
sorted_index=np.argsort(-pred.flatten())

In [93]:
outputs=[(pred.flatten()[idx], idx) for idx in sorted_index]

In [96]:
for i in outputs:
    print(le.inverse_transform([i[1]]))

['quality materials inc.']
['reliable supplies ltd.']
['eco friendly materials co.']
['global industries']
['acme corp']
